# Journals

This notebook is used for processing daily journals.

In [ ]:
import pandas as pd
import re
import datetime

# These will be specific to the journal data file used
JOURNAL_FILENAME = 'journals.txt'
HEADER_REGEX = r'(\d+)\/(\d+)\/(\d+)'
GRATEFUL_REGEX = r'\-(.*)'

Lets put the journals into two seperate dataframes. One will include the bulleted grateful items for that day. The other will include the raw paragraphs. 

In [ ]:
raw_paragraphs = []
raw_gratefuls = []

date = 'Empty'
grateful = 'Empty'
paragraph = 'Empty'

# Section headers (not to be included)
headers = ['\n', '#journal\n', 'Grateful:\n', '#daily\n',
           'Grateful List:\n', 'Successes:\n', 'Opportunities:\n', 
           'Grateful list:\n', ]

# Import journal data from text file
with open(JOURNAL_FILENAME) as file:
    for line in file:
        if 'Journal' in line:
            raw_date = re.search(HEADER_REGEX, line).groups()
            date = '/'.join(raw_date)
            weekday = datetime.datetime(int(raw_date[2]),
                                        int(raw_date[1]),
                                        int(raw_date[0])).weekday()
        elif line.startswith('-'):
            grateful = re.search(GRATEFUL_REGEX, line).group(1)
            raw_gratefuls.append((date, weekday, grateful))
        elif line not in headers:
            raw_paragraphs.append((date, weekday, line))
        else:
            pass
                
# Convert to dataframes
grateful = pd.DataFrame(raw_gratefuls, columns=['Date', 'Weekday', 'Grateful'])
paragraph = pd.DataFrame(raw_paragraphs, columns=['Date', 'Weekday', 'Line'])

In [ ]:
print(paragraph)

In [ ]:
grateful.head()
paragraph.head()